In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('data/kidney_disease.csv')  # Update path if needed

# Drop irrelevant columns
if 'id' in df.columns:
    df.drop(['id'], axis=1, inplace=True)

# Handle missing values
df.replace('?', np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)

# Convert object columns to numeric using LabelEncoder
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col].astype(str))

# Split features and target
X = df.drop('classification', axis=1)
y = df['classification']  # 1 for ckd, 0 for notckd (already encoded above)

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Save model and scaler
pickle.dump(model, open('kidney_model.pkl', 'wb'))
pickle.dump(scaler, open('kidney_scaler.pkl', 'wb'))

# Evaluation
y_pred = model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[52  0]
 [ 0 28]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        52
           1       1.00      1.00      1.00        28

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('data/parkinsons.csv')  # Update path if needed

# Drop 'name' column
df.drop(['name'], axis=1, inplace=True)

# Split features and target
X = df.drop('status', axis=1)
y = df['status']  # 1 = Parkinson’s, 0 = Healthy

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Save model and scaler
pickle.dump(model, open('parkinsons_model.pkl', 'wb'))
pickle.dump(scaler, open('parkinsons_scaler.pkl', 'wb'))

# Evaluation
y_pred = model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[ 5  2]
 [ 0 32]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.71      0.83         7
           1       0.94      1.00      0.97        32

    accuracy                           0.95        39
   macro avg       0.97      0.86      0.90        39
weighted avg       0.95      0.95      0.95        39



In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('data/indian_liver_patient.csv')  # Update path if needed

# Handle missing values
df.replace('?', np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)

# Encode 'Gender' column using LabelEncoder
le = LabelEncoder()
if 'Gender' in df.columns:
    df['Gender'] = le.fit_transform(df['Gender'].astype(str))

# Define target and encode it as binary
# 'Dataset' column: 1 = liver disease, 2 = no liver disease
df['Dataset'] = df['Dataset'].apply(lambda x: 1 if x == 1 else 0)

# Split features and target
X = df.drop('Dataset', axis=1)
y = df['Dataset']

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Save model and scaler
pickle.dump(model, open('liver_model.pkl', 'wb'))
pickle.dump(scaler, open('liver_scaler.pkl', 'wb'))

# Evaluation
y_pred = model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[15 15]
 [10 77]]
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.50      0.55        30
           1       0.84      0.89      0.86        87

    accuracy                           0.79       117
   macro avg       0.72      0.69      0.70       117
weighted avg       0.78      0.79      0.78       117



In [ ]:
# streamlit code 

import streamlit as st
import numpy as np
import pickle

# Load models and scalers
kidney_model = pickle.load(open('kidney_model.pkl', 'rb'))
kidney_scaler = pickle.load(open('kidney_scaler.pkl', 'rb'))

liver_model = pickle.load(open('liver_model.pkl', 'rb'))
liver_scaler = pickle.load(open('liver_scaler.pkl', 'rb'))

parkinsons_model = pickle.load(open('parkinsons_model.pkl', 'rb'))
parkinsons_scaler = pickle.load(open('parkinsons_scaler.pkl', 'rb'))

# Page title
st.title("🧬 Multiple Disease Prediction App")
st.sidebar.title("Choose Disease")

# Sidebar disease selection
disease = st.sidebar.selectbox("Select a Disease", ["Kidney Disease", "Liver Disease", "Parkinson's Disease"])

# ========== Helper Function ==========
def encode(val, mapping):
    return mapping.get(val, 0)

# ========== KIDNEY FORM ==========
if disease == "Kidney Disease":
    st.header("Kidney Disease Prediction")

    age = st.number_input("Age")
    bp = st.number_input("Blood Pressure")
    sg = st.number_input("Specific Gravity", format="%.2f")
    al = st.number_input("Albumin")
    su = st.number_input("Sugar")
    rbc = st.selectbox("Red Blood Cells", ["normal", "abnormal"])
    pc = st.selectbox("Pus Cell", ["normal", "abnormal"])
    pcc = st.selectbox("Pus Cell Clumps", ["notpresent", "present"])
    ba = st.selectbox("Bacteria", ["notpresent", "present"])
    bgr = st.number_input("Blood Glucose Random")
    bu = st.number_input("Blood Urea")
    sc = st.number_input("Serum Creatinine")
    sod = st.number_input("Sodium")
    pot = st.number_input("Potassium")
    hemo = st.number_input("Hemoglobin")
    pcv = st.number_input("Packed Cell Volume")
    wc = st.number_input("White Blood Cell Count")
    rc = st.number_input("Red Blood Cell Count")
    htn = st.selectbox("Hypertension", ["no", "yes"])
    dm = st.selectbox("Diabetes Mellitus", ["no", "yes"])
    cad = st.selectbox("Coronary Artery Disease", ["no", "yes"])
    appet = st.selectbox("Appetite", ["good", "poor"])
    pe = st.selectbox("Pedal Edema", ["no", "yes"])
    ane = st.selectbox("Anemia", ["no", "yes"])

    input_data = [
        age, bp, sg, al, su,
        encode(rbc, {"normal": 0, "abnormal": 1}),
        encode(pc, {"normal": 0, "abnormal": 1}),
        encode(pcc, {"notpresent": 0, "present": 1}),
        encode(ba, {"notpresent": 0, "present": 1}),
        bgr, bu, sc, sod, pot, hemo, pcv, wc, rc,
        encode(htn, {"no": 0, "yes": 1}),
        encode(dm, {"no": 0, "yes": 1}),
        encode(cad, {"no": 0, "yes": 1}),
        encode(appet, {"good": 0, "poor": 1}),
        encode(pe, {"no": 0, "yes": 1}),
        encode(ane, {"no": 0, "yes": 1})
    ]

    if st.button("Predict"):
        input_scaled = kidney_scaler.transform([input_data])
        result = kidney_model.predict(input_scaled)[0]
        prob = kidney_model.predict_proba(input_scaled)[0][1]
        st.success(f"Disease Prediction: {'Chronic Kidney Disease (CKD)' if result==1 else 'Normal'}")
        st.info(f"Probability of disease: {prob:.2f}")

# ========== LIVER FORM ==========
elif disease == "Liver Disease":
    st.header("Liver Disease Prediction")

    age = st.number_input("Age")
    gender = st.selectbox("Gender", ["Male", "Female"])
    tb = st.number_input("Total Bilirubin")
    db = st.number_input("Direct Bilirubin")
    alkphos = st.number_input("Alkaline Phosphotase")
    sgpt = st.number_input("Alamine Aminotransferase")
    sgot = st.number_input("Aspartate Aminotransferase")
    tp = st.number_input("Total Proteins")
    alb = st.number_input("Albumin")
    ag_ratio = st.number_input("A/G Ratio")

    gender_encoded = 1 if gender == "Male" else 0

    input_data = [age, gender_encoded, tb, db, alkphos, sgpt, sgot, tp, alb, ag_ratio]
    if st.button("Predict"):
        input_scaled = liver_scaler.transform([input_data])
        result = liver_model.predict(input_scaled)[0]
        prob = liver_model.predict_proba(input_scaled)[0][1]
        st.success(f"Disease Prediction: {'Liver Disease' if result==1 else 'Normal'}")
        st.info(f"Probability of disease: {prob:.2f}")

# ========== PARKINSON'S FORM ==========
elif disease == "Parkinson's Disease":
    st.header("Parkinson's Disease Prediction")

    fo = st.number_input("MDVP:Fo(Hz)")
    fhi = st.number_input("MDVP:Fhi(Hz)")
    flo = st.number_input("MDVP:Flo(Hz)")
    jitter_percent = st.number_input("MDVP:Jitter(%)")
    jitter_abs = st.number_input("MDVP:Jitter(Abs)")
    rap = st.number_input("MDVP:RAP")
    ppq = st.number_input("MDVP:PPQ")
    ddp = st.number_input("Jitter:DDP")
    shimmer = st.number_input("MDVP:Shimmer")
    shimmer_db = st.number_input("MDVP:Shimmer(dB)")
    apq3 = st.number_input("Shimmer:APQ3")
    apq5 = st.number_input("Shimmer:APQ5")
    apq = st.number_input("MDVP:APQ")
    dda = st.number_input("Shimmer:DDA")
    nhr = st.number_input("NHR")
    hnr = st.number_input("HNR")
    rpde = st.number_input("RPDE")
    dfa = st.number_input("DFA")
    spread1 = st.number_input("spread1")
    spread2 = st.number_input("spread2")
    d2 = st.number_input("D2")
    ppe = st.number_input("PPE")

    input_data = [fo, fhi, flo, jitter_percent, jitter_abs, rap, ppq, ddp, shimmer, shimmer_db,
                  apq3, apq5, apq, dda, nhr, hnr, rpde, dfa, spread1, spread2, d2, ppe]

    if st.button("Predict"):
        input_scaled = parkinsons_scaler.transform([input_data])
        result = parkinsons_model.predict(input_scaled)[0]
        prob = parkinsons_model.predict_proba(input_scaled)[0][1]
        st.success(f"Disease Prediction: {'Parkinson’s' if result==1 else 'Normal'}")
        st.info(f"Probability of disease: {prob:.2f}")
